In [ ]:
# Module imports

import database.connect
import pandas as pd
import numpy as np
import statistics
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Connect to SQL database

connection=database.connect.getConnection()
cursor=connection.cursor(buffered=True)

In [ ]:
categories = {'booking': ['booking', 'booked', 'book', 'ticket', 'tickets'],
            'canceling': ['canceled', 'cancellations'],
            'money': ['refund', 'compensation', 'claim', 'money', 'pay', 'paid'],
            'baggage': ['bag', 'baggage', 'luggage', 'bags'],
            'staff': ['staff', 'crew'],
            'waiting': ['waiting', 'delay', 'wait'],
            'boarding': ['boarding'],
            'stuck': ['stuck'],

            'information': ['info', 'information'],
            'customers': ['customer', 'customers', 'passenger', 'passengers'],
            'dm': [' dm'], 
            }

airlines_dict = {'KLM': ['klm'],
                'AirFrance':['airfrance',
                            'air france'],
                'British_Airways': ['british_airways',
                                    'british airways'],
                'AmericanAir': ['americanair',
                                'american airlines'],
                'Lufthansa': ['lufthansa'],
                'AirBerlin': ['airberlin',
                                'air berlin'],
                'AirBerlin assist': ['airberlin assist',
                                    'air berlin assist',
                                    'airberlinassist'],
                'easyJet': ['easyjet'],
                'RyanAir': ['ryanair'],
                'SingaporeAir': ['singaporeair',
                                'singapore airlines'],
                'Qantas': ['qantas'],
                'EtihadAirways': ['etihad airways',
                                'etihadairways',
                                'etihad'],
                'VirginAtlantic': ['virgin atlantic',
                                    'virginatlantic'],
            }

airlines_of_interest = ['AmericanAir', 'Other', 'British_Airways']

In [ ]:
def avg_resp_time():

    # Function takes in an airline and returns the mean sentiment

    Query= (f"""
                SELECT part_of.cID, COUNT(*), AVG(tweets.reply_time) 
                FROM tweets, part_of, conversations 
                WHERE tweets.id = part_of.tID AND part_of.cID = conversations.id AND conversations.Airline <> '0' 
                GROUP BY part_of.cID
                """)
    execute=cursor.execute(Query)
    result=cursor.fetchall()
    return result[0][0]

# TODO
def init_sent():

    # Find the median sentiment per airline
    try:
        Query = (f"""
                SELECT *
                FROM part_of, conversations
                WHERE conversations.Airline <> '0';
                 
                 """)
        execute = cursor.execute(Query)
        result = cursor.fetchall()
        
        return result[0][0]

    except:
        print("Error")
        return 0.00

def get_conversation_categories():
    excluided = ['dm', 'customers', 'information']
    relevant_categories = [key for key in categories.keys() if key not in excluided]
    # for each conversation, get weather it incluides a category.
    # returns a dataframe with the conversation id and the categorys
    categories_query = ""

    for category in relevant_categories:
        categories_query += f"(CASE WHEN tweets.{category} != 0 THEN 1 ELSE 0 END) AS {category}, "

    query = (f"""
                SELECT part_of.cID, {categories_query[:-2]}
                FROM tweets, part_of
                WHERE tweets.id = part_of.tID
                GROUP BY part_of.cID
                """)
    execute = cursor.execute(query)
    result = cursor.fetchall()
    df = pd.DataFrame(result)
    df.columns = ['cID'] + relevant_categories
    # remove the index column
    df = df.set_index('cID')
    return df
             

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



#file = 'https://aegis4048.github.io/downloads/notebooks/sample_data/unconv_MV_v5.csv'
def corr_matrix():
    df = pd.read_csv(file)

    df = df.iloc[:, 1:-1]

    corr = df.corr(method='spearman')

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize=(7, 6))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True, sep=100)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0, linewidths=.5)

    fig.suptitle('Correlation matrix of features', fontsize=15)


    fig.tight_layout()

In [ ]:
from sklearn import linear_model
file = 'https://aegis4048.github.io/downloads/notebooks/sample_data/unconv_MV_v5.csv'
def multiple_regress():
    df = pd.read_csv(file)

    #substitue for actual feature names later
    features = ['Por', 'Brittle', 'Perm', 'TOC']
    target = 'Prod'

    X = df[features].values.reshape(-1, len(features))
    y = df[target].values

    ols = linear_model.LinearRegression()
    model = ols.fit(X, y)

    #model.coef_

    print('Features                :  %s' % features)
    print('Regression Coefficients : ', [round(item, 2) for item in model.coef_])
    print('R-squared               :  %.2f' % model.score(X, y))
    print('Y-intercept             :  %.2f' % model.intercept_)
    print('')